<a href="https://colab.research.google.com/github/williamteles/Classification_Data-Mining/blob/main/Classification_Minera%C3%A7%C3%A3o_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas-profiling==2.7.1

!pip install pycaret

  Using cached pandas_profiling-2.7.1-py2.py3-none-any.whl (252 kB)
  Using cached visions-0.4.1-py3-none-any.whl (58 kB)
  Attempting uninstall: visions
    Found existing installation: visions 0.7.4
    Uninstalling visions-0.7.4:
      Successfully uninstalled visions-0.7.4
  Attempting uninstall: pandas-profiling
    Found existing installation: pandas-profiling 3.1.0
    Uninstalling pandas-profiling-3.1.0:
      Successfully uninstalled pandas-profiling-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 2.3.5 requires pandas-profiling>=2.8.0, but you have pandas-profiling 2.7.1 which is incompatible.
  Using cached pandas_profiling-3.1.0-py2.py3-none-any.whl (261 kB)
  Using cached visions-0.7.4-py3-none-any.whl (102 kB)
  Attempting uninstall: visions
    Found existing installation: visions 0.4.1
    Uninstalling visions-0.4.1:
      Successf

In [2]:
import pandas as pd
import numpy as np
import pandas_profiling
from sklearn import model_selection
from sklearn.utils import resample
from sklearn.preprocessing import scale
from pycaret.classification import setup
from pycaret.classification import compare_models

In [3]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00571/hcvdat0.csv"
df = pd.read_csv(url, delimiter=',', index_col=0)
df.head()

,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
1,0=Blood Donor,32,m,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
2,0=Blood Donor,32,m,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
3,0=Blood Donor,32,m,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
4,0=Blood Donor,32,m,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
5,0=Blood Donor,32,m,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7


In [4]:
categories = df.Category.unique()
categories

array(['0=Blood Donor', '0s=suspect Blood Donor', '1=Hepatitis',
       '2=Fibrosis', '3=Cirrhosis'], dtype=object)

In [5]:
df.Category.replace(categories, [0,0,1,1,1], inplace=True)


In [6]:
df['Sex'] = df['Sex'].map({'m': 0, 'f': 1})
df

,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
1,0,32,0,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
2,0,32,0,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
3,0,32,0,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3
4,0,32,0,43.2,52.0,30.6,22.6,18.9,7.33,4.74,80.0,33.8,75.7
5,0,32,0,39.2,74.1,32.6,24.8,9.6,9.15,4.32,76.0,29.9,68.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,1,62,1,32.0,416.6,5.9,110.3,50.0,5.57,6.30,55.7,650.9,68.5
612,1,64,1,24.0,102.8,2.9,44.4,20.0,1.54,3.02,63.0,35.9,71.3
613,1,64,1,29.0,87.3,3.5,99.0,48.0,1.66,3.63,66.7,64.2,82.0
614,1,46,1,33.0,NaN,39.0,62.0,20.0,3.56,4.20,52.0,50.0,71.0


In [7]:
df.isnull().sum()

Category     0
Age          0
Sex          0
ALB          1
ALP         18
ALT          1
AST          0
BIL          0
CHE          0
CHOL        10
CREA         0
GGT          0
PROT         1
dtype: int64

In [8]:
df = df.dropna()
df.shape

(589, 13)

In [9]:
df.describe()

,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
count,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000
mean,0.095076,47.417657,0.383701,41.624278,68.123090,26.575382,33.772835,11.018166,8.203633,5.391341,81.669100,38.198472,71.890153
std,0.293570,9.931334,0.486700,5.761794,25.921072,20.863120,32.866871,17.406572,2.191073,1.128954,50.696991,54.302407,5.348883
min,0.000000,23.000000,0.000000,14.900000,11.300000,0.900000,10.600000,0.800000,1.420000,1.430000,8.000000,4.500000,44.800000
25%,0.000000,39.000000,0.000000,38.800000,52.500000,16.400000,21.500000,5.200000,6.930000,4.620000,68.000000,15.600000,69.300000
50%,0.000000,47.000000,0.000000,41.900000,66.200000,22.700000,25.700000,7.100000,8.260000,5.310000,77.000000,22.800000,72.100000
75%,0.000000,54.000000,1.000000,45.100000,79.900000,31.900000,31.700000,11.000000,9.570000,6.080000,89.000000,37.600000,75.200000
max,1.000000,77.000000,1.000000,82.200000,416.600000,325.300000,324.000000,209.000000,16.410000,9.670000,1079.100000,650.900000,86.500000


In [10]:
def limitar_anomalias(data, anomalia_cols):
  for col in anomalia_cols:
    q25, q75 = np.percentile(data[col], 25), np.percentile(data[col], 75)

    iiq = q75 - q25

    print(f'Percentis coluna {col}: 25%={q25:.3f}, 75%={q75:.3f}, IIQ={iiq:.3f}')

    corte = iiq * 1.5

    inferior, superior = q25 - corte, q75 + corte

    anomalias = [x for x in data[col] if x < inferior or x > superior]

    print(f'Outliers Identificados na coluna {col}: {len(anomalias)}')

    data[col] = np.where(data[col] > superior, superior, 
                         np.where(data[col] < inferior, inferior, 
                                  data[col]))
  
  return data


def normalizar(data, normalizar_cols):
  for col in normalizar_cols:
    data[col] = (data[col] - data[col].min()) / (data[col].max() - data[col].min())

  return data

In [11]:
anomalia_cols = ['ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT']
df = limitar_anomalias(df.copy(), anomalia_cols)

df.describe()

Percentis coluna ALB: 25%=38.800, 75%=45.100, IIQ=6.300
Outliers Identificados na coluna ALB: 27
Percentis coluna ALP: 25%=52.500, 75%=79.900, IIQ=27.400
Outliers Identificados na coluna ALP: 10
Percentis coluna ALT: 25%=16.400, 75%=31.900, IIQ=15.500
Outliers Identificados na coluna ALT: 26
Percentis coluna AST: 25%=21.500, 75%=31.700, IIQ=10.200
Outliers Identificados na coluna AST: 54
Percentis coluna BIL: 25%=5.200, 75%=11.000, IIQ=5.800
Outliers Identificados na coluna BIL: 47
Percentis coluna CHE: 25%=6.930, 75%=9.570, IIQ=2.640
Outliers Identificados na coluna CHE: 23
Percentis coluna CHOL: 25%=4.620, 75%=6.080, IIQ=1.460
Outliers Identificados na coluna CHOL: 12
Percentis coluna CREA: 25%=68.000, 75%=89.000, IIQ=21.000
Outliers Identificados na coluna CREA: 12
Percentis coluna GGT: 25%=15.600, 75%=37.600, IIQ=22.000
Outliers Identificados na coluna GGT: 69
Percentis coluna PROT: 25%=69.300, 75%=75.200, IIQ=5.900
Outliers Identificados na coluna PROT: 19


,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
count,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000
mean,0.095076,47.417657,0.383701,41.698048,67.207640,25.063667,27.840747,8.759083,8.206723,5.383582,78.629032,30.029372,72.072581
std,0.293570,9.931334,0.486700,4.967393,20.036769,12.474262,8.941355,4.954285,2.062554,1.095048,15.296737,19.639276,4.712771
min,0.000000,23.000000,0.000000,29.350000,11.400000,0.900000,10.600000,0.800000,2.970000,2.430000,36.500000,4.500000,60.450000
25%,0.000000,39.000000,0.000000,38.800000,52.500000,16.400000,21.500000,5.200000,6.930000,4.620000,68.000000,15.600000,69.300000
50%,0.000000,47.000000,0.000000,41.900000,66.200000,22.700000,25.700000,7.100000,8.260000,5.310000,77.000000,22.800000,72.100000
75%,0.000000,54.000000,1.000000,45.100000,79.900000,31.900000,31.700000,11.000000,9.570000,6.080000,89.000000,37.600000,75.200000
max,1.000000,77.000000,1.000000,54.550000,121.000000,55.150000,47.000000,19.700000,13.530000,8.270000,120.500000,70.600000,84.050000


In [12]:
normalizar_cols = ['ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT']

df = normalizar(df.copy(), anomalia_cols)

df.describe()

,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
count,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000,589.000000
mean,0.095076,47.417657,0.383701,0.490002,0.509194,0.445413,0.473647,0.421116,0.495902,0.505750,0.501536,0.386223,0.492482
std,0.293570,9.931334,0.486700,0.197119,0.182817,0.229940,0.245642,0.262131,0.195318,0.187508,0.182104,0.297115,0.199694
min,0.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,39.000000,0.000000,0.375000,0.375000,0.285714,0.299451,0.232804,0.375000,0.375000,0.375000,0.167927,0.375000
50%,0.000000,47.000000,0.000000,0.498016,0.500000,0.401843,0.414835,0.333333,0.500947,0.493151,0.482143,0.276853,0.493644
75%,0.000000,54.000000,1.000000,0.625000,0.625000,0.571429,0.579670,0.539683,0.625000,0.625000,0.625000,0.500756,0.625000
max,1.000000,77.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
duplicados = df.duplicated()
print(duplicados.any())

False


In [14]:
mascara = df.Category == 0
df_0 = df[mascara]
df_1 = df[~mascara]
df_oversample = resample(df_1, n_samples=len(df_0), random_state=13)
df2 = pd.concat([df_0, df_oversample])
df2.Category.value_counts()

1    533
0    533
Name: Category, dtype: int64

In [15]:
clf = setup(data=df2, target='Category', silent=True, verbose=False, log_experiment=True)
best = compare_models()
print(best)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9987,1.0000,1.0000,0.9974,0.9987,0.9973,0.9974,0.517
ada,Ada Boost Classifier,0.9987,1.0000,1.0000,0.9974,0.9987,0.9973,0.9974,0.117
et,Extra Trees Classifier,0.9986,1.0000,1.0000,0.9974,0.9987,0.9973,0.9973,0.466
lightgbm,Light Gradient Boosting Machine,0.9932,0.9999,1.0000,0.9871,0.9934,0.9865,0.9868,0.117
gbc,Gradient Boosting Classifier,0.9919,1.0000,1.0000,0.9845,0.9921,0.9838,0.9842,0.167
dt,Decision Tree Classifier,0.9866,0.9865,1.0000,0.9747,0.9870,0.9731,0.9739,0.016
qda,Quadratic Discriminant Analysis,0.9732,0.9910,0.9707,0.9769,0.9733,0.9464,0.9473,0.016
lr,Logistic Regression,0.9718,0.9953,0.9707,0.9740,0.9719,0.9436,0.9445,0.333
ridge,Ridge Classifier,0.9517,0.0000,0.9227,0.9808,0.9504,0.9035,0.9059,0.014
lda,Linear Discriminant Analysis,0.9490,0.9925,0.9200,0.9783,0.9477,0.8981,0.9006,0.015


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=3965, verbose=0,
                       warm_start=False)
